In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from itertools import product
from collections import defaultdict

from IPython.display import display
import time

In [2]:
new_nyc_df = pd.read_csv('cleaned_NYC_records_duration.csv')
new_nyc_df['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df["UTC_time"])
less_equal_2hours_df = new_nyc_df[(new_nyc_df.duration <= 2) & (new_nyc_df.duration != -1)]

months = [4, 5, 6]
less_than_2hrs_3months_df = less_equal_2hours_df.copy()
less_than_2hrs_3months_df['month'] = less_than_2hrs_3months_df['cleaned_UTC_time'].dt.month
less_than_2hrs_3months_df = less_than_2hrs_3months_df[less_than_2hrs_3months_df['month'].isin(months)]

In [3]:
def finding_days_users_less_than_2hrs_records_groupto(df):
    all_users_days_df = df.copy()
    all_users_days_df['date'] = all_users_days_df.apply(lambda x : x.cleaned_UTC_time.date(), axis=1)
    all_users_unique_days_df = all_users_days_df.groupby('user_id')['date'].nunique().reset_index(name='counts')
    all_users_unique_days_df = all_users_unique_days_df.sort_values(by=['counts'])
    return all_users_unique_days_df

In [4]:
user_days_df = finding_days_users_less_than_2hrs_records_groupto(less_than_2hrs_3months_df)
frequent_users_df = user_days_df[user_days_df['counts'] >= 50]

In [5]:
user_set_df = less_than_2hrs_3months_df[less_than_2hrs_3months_df.user_id.isin(frequent_users_df.user_id)]

# Eateries - E
# Others - O
# Outdoors - Out
# Education - Edu
# Shops - Sh
# Arts - A
# Services - Ser
# Religious Places - R
# Entertainment - Ent
# Exhibitions - Ex
# Fitness - F

general_categories_tags_dict = {
        'Eateries': 'E',
        'Others': 'O',
        'Outdoor': 'Out',
        'Education': 'Edu',
        'Shops': 'Sh',
        'Arts': 'A',
        'Services': 'Ser',
        'Religious Places': 'R',
        'Entertainment': 'Ent',
        'Exhibitions': 'Ex',
        'Fitness': 'F'
    }

def generate_general_categories_tags(row):
    return general_categories_tags_dict[row['general_venue_category_name']]

user_set_df['general_venue_category_tag'] = user_set_df.apply(generate_general_categories_tags, axis=1)

<ipython-input-5-68f05843e2db>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_set_df['general_venue_category_tag'] = user_set_df.apply(generate_general_categories_tags, axis=1)


In [6]:
user_set_df

,user_id,venue_id,venue_category_id,venue_category_name,general_venue_category_name,latitude,longitude,UTC_time,duration,cleaned_UTC_time,month,general_venue_category_tag
15,445,4b9830a5f964a520c73235e3,4bf58dd8d48988d118951735,Food & Drink Shop,Eateries,40.826790,-73.949509,Tue Apr 03 18:10:39 +0000 2012,0.557778,2012-04-03 18:10:39+00:00,4,E
25,443,4a676ff4f964a5206fc91fe3,4bf58dd8d48988d124941735,Office,Others,40.655535,-74.008629,Tue Apr 03 18:14:44 +0000 2012,1.390556,2012-04-03 18:14:44+00:00,4,O
31,962,40b68100f964a5207d001fe3,4bf58dd8d48988d163941735,Park,Outdoor,40.742188,-73.987924,Tue Apr 03 18:17:17 +0000 2012,0.034444,2012-04-03 18:17:17+00:00,4,Out
47,768,4dbdebed6e810768bf82ad95,4bf58dd8d48988d1c9941735,Ice Cream Shop,Eateries,40.803965,-74.005135,Tue Apr 03 18:33:29 +0000 2012,0.008611,2012-04-03 18:33:29+00:00,4,E
49,768,4e79392ce4cdb158f1cf4b75,4bf58dd8d48988d103941735,Home,Others,40.802042,-74.000633,Tue Apr 03 18:34:00 +0000 2012,1.926944,2012-04-03 18:34:00+00:00,4,O
...,...,...,...,...,...,...,...,...,...,...,...,...
104376,293,4dd10cf4b0fb25f6e3817b33,4bf58dd8d48988d179941735,Bagel Shop,Eateries,40.714049,-73.949311,Sat Jun 30 23:30:37 +0000 2012,0.015833,2012-06-30 23:30:37+00:00,6,E
104377,293,4b12f7c7f964a520cf9123e3,4bf58dd8d48988d1ca941735,Pizza Place,Eateries,40.714160,-73.949383,Sat Jun 30 23:31:34 +0000 2012,0.106944,2012-06-30 23:31:34+00:00,6,E
104395,120,4d40485889616dcb9cf002b5,4bf58dd8d48988d174941735,Office,Others,40.688074,-73.981043,Sat Jun 30 23:45:14 +0000 2012,0.030556,2012-06-30 23:45:14+00:00,6,O
104406,84,4d77e2e23915721ec6580783,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Others,40.767918,-73.915972,Sat Jun 30 23:53:22 +0000 2012,0.022500,2012-06-30 23:53:22+00:00,6,O


In [7]:
subsequence_all_df = user_set_df.copy()
subsequence_all_df['date'] = subsequence_all_df.apply(lambda x : x.cleaned_UTC_time.date(), axis=1)
subsequence_all_df = subsequence_all_df.groupby(['user_id', 'date'])['general_venue_category_tag'].apply(lambda x: list(x)).reset_index(name='category_set')
subsequence_all_df['counts'] = subsequence_all_df['category_set'].apply(lambda x: len(x))

In [8]:
subsequence_all_df

,user_id,date,category_set,counts
0,84,2012-04-03,[E],1
1,84,2012-04-04,"[Out, E, A, A, O, O, A, E, E, Sh, E]",11
2,84,2012-04-07,"[A, O, O, E]",4
3,84,2012-04-08,"[E, E, Ent, O, Ex, O, Ent, Out, Ent, Out, E, E...",19
4,84,2012-04-09,"[F, E, A, Out, Out, O, E, Ser, Ser, F, E]",11
...,...,...,...,...
2002,990,2012-06-14,"[Ser, Ser, Ser, Ser, Ser, O]",6
2003,990,2012-06-17,"[Ser, Ser, Ser, Ser, Ser]",5
2004,990,2012-06-18,[O],1
2005,990,2012-06-21,"[Ser, O]",2


In [13]:
def mine_rec(patt, mdb, db, results, minsup):
    def localOccurs(mdb):
        occurs = defaultdict(list)

        # The stoppos is always start with 1, and 1 represents the 0th index element in the sequence
        for (i, stoppos) in mdb:
            seq = db[i]
            for j in range(stoppos, len(seq)):
                l = occurs[seq[j]]
                if len(l) == 0 or l[-1][0] != i:
                    l.append((i, j + 1))

        return occurs

    for (c, newmdb) in localOccurs(mdb).items():
        newsup = len(newmdb)

        if newsup >= minsup:
            newpatt = patt + [c]

            results.append((newpatt, [i for (i, stoppos) in newmdb]))
            mine_rec(newpatt, newmdb, db, results, minsup)

In [14]:
def mine_specific_user_rec(user_id):
    db = np.array(subsequence_all_df[(subsequence_all_df.user_id == user_id)].category_set)
    minsup = len(db)//2
    results = []
    
    mine_rec([], [(i, 0) for i in range(len(db))], db, results, minsup)
    return results

In [11]:
# db = np.array(subsequence_all_df[(subsequence_all_df.user_id == 280)].category_set)

# minsup = len(db)//2
# results = []

# mine_rec([], [(i, 0) for i in range(len(db))])

In [18]:
results_280 = mine_specific_user_rec(280)
result_280 = [{'sequence': i[0], 'support': len(i[1])} for i in results_280]
result_280

[{'sequence': ['E'], 'support': 44},
 {'sequence': ['E', 'E'], 'support': 35},
 {'sequence': ['E', 'E', 'E'], 'support': 26}]

In [19]:
results_962 = mine_specific_user_rec(962)
result_962 = [{'sequence': i[0], 'support': len(i[1])} for i in results_962]
result_962

[{'sequence': ['O'], 'support': 39}]

In [20]:
results_178 = mine_specific_user_rec(178)
result_178 = [{'sequence': i[0], 'support': len(i[1])} for i in results_178]
result_178

[{'sequence': ['E'], 'support': 43},
 {'sequence': ['E', 'E'], 'support': 28},
 {'sequence': ['E', 'Sh'], 'support': 27},
 {'sequence': ['Sh'], 'support': 39},
 {'sequence': ['Sh', 'O'], 'support': 29},
 {'sequence': ['O'], 'support': 37}]